# Exercise 6
## Solution by Anton Wiehe & Angelie Kraft

In [ ]:
%matplotlib notebook
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy.stats as st
import sys

In [ ]:
gaussian_a = np.random.multivariate_normal([0, -2], [[1, 0], 
                                                    [0, 1]], 100)

gaussian_b = np.random.multivariate_normal([0, 3], [[2, 0], 
                                                    [3, 2]], 100)

gaussian_c = np.random.multivariate_normal([7, 0], [[2, 0], 
                                                    [1, 3]], 100)

plt.scatter(gaussian_a[:, 0], gaussian_a[:, 1], color='blue')
plt.scatter(gaussian_b[:, 0], gaussian_b[:, 1], color='green')
plt.scatter(gaussian_c[:, 0], gaussian_c[:, 1], color='red')
plt.show()

In [ ]:
data = np.concatenate([gaussian_a, gaussian_b, gaussian_c], axis=0)
data.shape

In [ ]:
class EM:
    def __init__(self, num_gaussians=3):
        self.num_gaussians = num_gaussians
        self.phis = np.ones(self.num_gaussians) / self.num_gaussians
        self.fig = plt.figure()
        self.ax = plt.gca()
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
        
        
    def fit(self, X, y, steps=10, early_stopping=False):
        self.data_dim = len(X[0])
        self.m = len(X)
        # Initialize mus:
        self.mu_idxs = np.random.randint(self.m, size=self.num_gaussians)
        self.mus = X[self.mu_idxs]
        self.vars = np.ones((self.num_gaussians, self.data_dim)) # use std of every dim to initialize
        
        for i in range(steps):
            self.plot(X)

            last_vars = self.vars
            
            ws, probs = self.e_step(X)
            self.m_step(X, ws)
            
            
            log_probs = np.log(probs)
            log_llh = np.sum(log_probs) * -1
                  
            #print("vars: ", self.vars)
            print("Step ", i)
            #print("diff in vars: ", np.array(self.vars) - np.array(last_vars))
            print("log_llh: ", log_llh)
            print()
            if early_stopping and np.sum(np.abs(self.vars - last_vars)) < 0.1:
                print("Stopped at ", i)
                return self
            

            
        print("Stopped after ", steps, " steps.")    
        return self
    
    def predict(self, X):
        return self.e_step(X)
    
    def plot(self, X):
        self.ax.clear()
        colors = np.array([(1, 0, 0), (0, 1, 0), (0, 0, 1), (1, 1, 0), (1, 0, 1), (0, 1, 1)])
        assignments, probs = self.e_step(X)
        assignment_colors = np.zeros((len(X), 3))
        x = np.linspace(-6, 11, 50)
        y = np.linspace(-6, 8, 50)

        X_lin_orig, Y_lin_orig = np.meshgrid(x, y)
        X_lin, Y_lin = X_lin_orig.flatten(), Y_lin_orig.flatten()
        concat_X_Y = np.stack((X_lin, Y_lin), axis=1)
    
        Z = np.zeros((self.num_gaussians, 50, 50))
        probs = self.get_prob(concat_X_Y)
        for i in range(self.num_gaussians):
            Z[i] = probs[:, i].reshape((50, 50))

        for i in range(self.num_gaussians):
            assignment_colors += np.reshape(assignments[:, i], (len(X), 1)) * colors[i]
            self.ax.contour(X_lin_orig, Y_lin_orig, Z[i], levels=3)
        assignment_colors = np.clip(assignment_colors, 0, 1)
        #summed_Z = np.sum(Z, axis=0) 
        #self.ax.contour(X_lin_orig, Y_lin_orig, summed_Z, levels=5)
        self.ax.scatter(X[:, 0], X[:, 1], color=assignment_colors)
        self.fig.canvas.draw()      
        plt.pause(0.01)
            

    def e_step(self, data):
        ws = np.zeros((len(data), self.num_gaussians))
        ps = np.zeros((len(data), self.num_gaussians))
        for i, x in enumerate(data):
            p_xs = np.array([st.multivariate_normal.pdf(x, mean=self.mus[k], cov=self.vars[k]) * self.phis[k] 
                            for k in range(self.num_gaussians)])
            ps[i] = p_xs
            ws[i] = p_xs / np.sum(p_xs)
        return ws, ps
    
    def get_prob(self, data):
        ws = np.zeros((len(data), self.num_gaussians))
        for i, x in enumerate(data):
            p_xs = np.array([st.multivariate_normal.pdf(x, mean=self.mus[k], cov=self.vars[k]) * self.phis[k] 
                            for k in range(self.num_gaussians)])
            ws[i] = p_xs
        return ws
    
    def m_step(self, data, ws):
        m = len(data)
        self.phis = np.sum(ws, axis=0) / m
        for k in range(self.num_gaussians):
            self.mus[k] = np.sum(ws[:, k].reshape(m, 1) * data, axis=0) / (self.phis[k] * m)
            self.vars[k] = np.sum(ws[:, k].reshape(m, 1) * (data - self.mus[k])**2, axis=0) / (self.phis[k] * m)

In [ ]:
em_clusterer = EM(num_gaussians=4)
em_clusterer.fit(data, None, steps=20)
#em_clusterer.plot(data)
#em_clusterer.predict(data)

In [ ]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=4)
clf.fit(data)
predicted_labels = clf.predict(data)
all_colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (1, 1, 0), (1, 0, 1), (0, 1, 1)]
colors = [all_colors[label] for label in predicted_labels]

In [ ]:
print(predicted_labels)

In [ ]:
plt.ion()
plt.figure(0)
plt.scatter(data[:, 0], data[:, 1], color=colors)
centers = np.array(clf.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], marker='x', color='black', s=250, linewidth=4)
plt.show()

In [ ]:
clf.cluster_centers_